<div style="display: flex; background-color: RGB(100,100,100);" >
  <h1 style="margin: auto; padding: 20px; "> Projet: ANALYSE DES VENTES D'UNE LIBRAIRIES! </h1>
</div>

### Sommaire
###  <a href="#AF"> 1. Analyse de la forme des fichiers et regularisation en vu des prochaines analyses </a>
###  <a href="#F"> 2. Analyse des indicateurs de ventes </a>
###  <a href="#FA"> 3. Analyse bivariee et tests statistiques </a>

## Mission du projet

<div style="border: 1px solid RGB(100,100,100);" >
    Lapage a un site de vente en ligne depuis 2 ans et ajourd'hui, on souhaite pouvoir analyser ses points forts, ses points faibles, les comportements clients, etc. Afin de décider par exemple si nous devons créer certaines offres, adapter certains prix, etc. Cette analyse va être découpée en deux parties:
    <ul>
        <li> Une analyse des différents indicateurs de vente (Différents indicateurs et graphiques autour du
chiffre d'affaires. Evolution dans le temps du CA et mise en place d'une décomposition en moyenne mobile pour évaluer la tendance globale, faire un zoom sur les références, pour voir un peu les tops et les flops, la répartition par catégorie, etc.) ; </li>
        <li> Une analyse plus ciblée sur les clients : l’objectif serait cette fois-ci de comprendre le comportement de nos clients en ligne (le lien entre le genre d’un client et les catégories des livres achetés ; et ensuite le lien entre l’âge des clients et le montant total des achats, la fréquence d’achat, la taille du panier moyen et les catégories des livres achetés.) </li>
    </ul>
</div>

<div style="border: 1px solid RGB(100,100,100);" >
    <h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); "> <a id="AF"> 1 - Analyse de la forme des fichiers et regularisation en vu des prochaines analyses </a></h3>
</div>

In [1]:
# Importation des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
import scipy
import statsmodels.api as sm
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy.stats import pearsonr

In [6]:
# Importation des fichiers csv sous forme de dataframes
customer = pd.read_csv('customers.csv')
product = pd.read_csv('products.csv')
transaction = pd.read_csv('transactions.csv')

### Exploration du fichier des clients

In [8]:
display(customer.head())
display(customer.dtypes)
display(customer.shape)
display(customer.isna().sum())
display(customer.nunique())
display(customer.describe())

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


client_id    object
sex          object
birth         int64
dtype: object

(8623, 3)

client_id    0
sex          0
birth        0
dtype: int64

client_id    8623
sex             2
birth          76
dtype: int64

,birth
count,8623.000000
mean,1978.280877
std,16.919535
min,1929.000000
25%,1966.000000
50%,1979.000000
75%,1992.000000
max,2004.000000


#### Observation:
* On compte 8623 clients carracterises par le sex et la date de naissance ;
* Type variable attendu ;
* Aucune valeur manquante ; 
* Aucune valeur aberrante.


client_id => Primary Key

### Exploration du fichier des produits

In [9]:
display(product.head())
display(product.dtypes)
display(product.shape)
display(product.isna().sum())
display(product.nunique())
display(product.describe())

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


id_prod     object
price      float64
categ        int64
dtype: object

(3287, 3)

id_prod    0
price      0
categ      0
dtype: int64

id_prod    3287
price      1455
categ         3
dtype: int64

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


#### Observation:
* Le fichier comporte 3287 produits avec le prix unitaire et la categorie associee (3) ;
* Type variable attendu, pour faciliter les analyses, convertir 'categ' en object ;
* Aucune valeur manquante ;
* Valeurs aberrantes sur prix; prix minimum (-1), piste à explorer ;
* Trois valeurs uniques pour 'catégories' ;
* A première vue, le premier chiffre de 'id_prod' correspond à la 'catégorie', théorie à vérifier.


id_prod => Primary Key

In [10]:
# Convertir variable 'categ' en string
product['categ'] = product['categ'].astype(str)
# Chercher les produits dont le prix est 0 ou moins
product.loc[product['price']<=0]

,id_prod,price,categ
731,T_0,-1.0,0


### Exploration du fichier des transaction(commandes)

In [11]:
display(transaction.head())
display(transaction.dtypes)
display(transaction.shape)
display(transaction.isna().sum())
display(transaction.nunique())
display(transaction.describe())

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


id_prod       object
date          object
session_id    object
client_id     object
dtype: object

(679532, 4)

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

id_prod         3267
date          679371
session_id    342316
client_id       8602
dtype: int64

,id_prod,date,session_id,client_id
count,679532,679532,679532,679532
unique,3267,679371,342316,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,2252,13,200,25488


#### Observation:
* Variable 'date' de mauvais type
* Aucune valeurs manquante
* Nombre de valeurs uniques pour 'date' et 'cession' étranges, aucun ne correspond au nombre de lignes total de la dataframe. Indique des doublons. 

#### Action à prendre: 
* Vérifier variable 'date'
* Explorer les variables 'date' et 'session_id'

In [12]:
# Lignes contenant 'test' dans variable 'date'
test = transaction.loc[transaction['date'].str.contains('test')].sort_values('date')
display(test)
display(test.nunique())

,id_prod,date,session_id,client_id
96687,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
453857,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
91133,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
607892,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_0
548383,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1
...,...,...,...,...
392443,T_0,test_2021-03-01 02:30:02.237448,s_0,ct_0
19312,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0
670680,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1
573155,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0


id_prod        1
date          39
session_id     1
client_id      2
dtype: int64

#### Observation:
Tous les tests ont été faits le même jours à des fractions de seconde d'intervalles et ce sont les premières transactions faites sur le même 'session_id'. 
Ces transactions sont les ventes du même produit (T_0) qui correspondent au produit donc le prix est à -1 qu'on avait vu pendant le nettoyage de la dataframe 'produit'.
Nous observons que ces transaction sont faîtes par seulement deux clients (ct_0 et ct_1) .  

Ce sont sûrement des tests que l'entreprise a fait pour vérifier le fonctionnement du site. 
Ct qui pourrait indiquer 'client test', T_0 qui pourrait indiquer Produit test. 


In [15]:
# Filtrage des donnees correspondant aux tests effectues
product_test = product['id_prod'].str.contains('T')
customer_test = customer['client_id'].str.contains('ct')
transaction_test = transaction['date'].str.contains('test')

In [16]:
# Copie des donnees sauf les donnees correspondant aux tests
transaction = transaction.loc[~transaction_test].copy()
product = product.loc[~product_test].copy()
customer = customer.loc[~customer_test].copy()

In [17]:
# Conversion variable 'date' en type date. 
transaction['date'] = pd.to_datetime(transaction['date'])

In [18]:
# Vérification des changements sur le dataframe des transactions
display(transaction.head())
display(transaction.shape)
display(transaction.dtypes)
display(transaction.nunique())

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


(679332, 4)

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object

id_prod         3266
date          679332
session_id    342315
client_id       8600
dtype: int64

### Fusion des dataframes
#### Customer et transaction

In [20]:
# Outter merge des dataframes customers et transaction
custo_trans = pd.merge(customer, transaction, on='client_id', how='outer', indicator=True )
display(custo_trans['_merge'].unique())
display(custo_trans.shape)
display(custo_trans.head())

['both', 'left_only']
Categories (3, object): ['left_only', 'right_only', 'both']

(679353, 7)

,client_id,sex,birth,id_prod,date,session_id,_merge
0,c_4410,f,1967,0_1316,2021-12-29 09:11:18.860592,s_141762,both
1,c_4410,f,1967,1_385,2021-03-22 01:40:22.782925,s_9707,both
2,c_4410,f,1967,1_190,2021-11-12 18:11:43.280574,s_118628,both
3,c_4410,f,1967,0_1455,2021-03-22 14:29:25.189266,s_9942,both
4,c_4410,f,1967,1_483,2022-03-13 21:35:55.949042,s_178686,both


**Observation**: 
Certains 'client_id' sont dans la dataframe 'customer' mais pas dans la dataframe 'transaction', ce qui veut dire que nous avions des clients qui n'ont pas fait de transactions.

#### Produit et transaction

In [21]:
# Outter merge des dataframe produit et transaction
product_trans = pd.merge(transaction, product, on='id_prod', how='outer', indicator=True )
display(product_trans['_merge'].unique())
display(product_trans.shape)
display(product_trans.head())

['both', 'left_only', 'right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

(679353, 7)

,id_prod,date,session_id,client_id,price,categ,_merge
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,4.18,0,both
1,0_1518,2021-09-26 12:37:29.780414,s_95811,c_6197,4.18,0,both
2,0_1518,2021-05-06 17:14:43.117440,s_30782,c_682,4.18,0,both
3,0_1518,2022-03-16 18:57:10.420103,s_180057,c_5932,4.18,0,both
4,0_1518,2022-11-12 18:58:10.574853,s_296584,c_7217,4.18,0,both


In [22]:
# Produit qui sont dans dataframe'transaction' mais pas dans dataframe 'produit'
left_only = product_trans.loc[product_trans['_merge']=='left_only']
display(left_only['id_prod'].unique())
display(left_only.shape)
display(left_only)

array(['0_2245'], dtype=object)

(221, 7)

,id_prod,date,session_id,client_id,price,categ,_merge
542560,0_2245,2022-09-23 07:22:38.636773,s_272266,c_4746,NaN,NaN,left_only
542561,0_2245,2022-07-23 09:24:14.133889,s_242482,c_6713,NaN,NaN,left_only
542562,0_2245,2022-12-03 03:26:35.696673,s_306338,c_5108,NaN,NaN,left_only
542563,0_2245,2021-08-16 11:33:25.481411,s_76493,c_1391,NaN,NaN,left_only
542564,0_2245,2022-07-16 05:53:01.627491,s_239078,c_7954,NaN,NaN,left_only
...,...,...,...,...,...,...,...
542776,0_2245,2021-08-25 09:06:03.504061,s_80395,c_131,NaN,NaN,left_only
542777,0_2245,2022-03-06 19:59:19.462288,s_175311,c_4167,NaN,NaN,left_only
542778,0_2245,2022-05-16 11:35:20.319501,s_209381,c_4453,NaN,NaN,left_only
542779,0_2245,2022-02-11 09:05:43.952857,s_163405,c_1098,NaN,NaN,left_only


**Observation**: 
Le produit '0_2245' a été vendu mais il n'est pas présent dans la dataframe 'produit', donc nous n'avions pas les information sur ce produit, il faudrait trouver ces information car il a été vendu 221 fois ce qui pourrait avoir une grande influence sur l'analyse.

In [24]:
# Calculer le prix moyen des articles de catégorie '0'
product_cat_0 = product.loc[product['categ']=='0']
moyen_price_cat_0 = product_cat_0['price'].mean()
# Ajouter une nouvelle ligne à la dataframe 'product' contenant les information du produit '0_2245'
product.loc[-1] = ['0_2245',moyen_price_cat_0,'0']
product.reset_index(drop=True)

,id_prod,price,categ
0,0_1421,19.990000,0
1,0_1368,5.130000,0
2,0_731,17.990000,0
3,1_587,4.990000,1
4,0_1507,3.990000,0
...,...,...,...
3282,0_146,17.140000,0
3283,0_802,11.220000,0
3284,1_140,38.560000,1
3285,0_1920,25.160000,0


In [26]:
# Produit qui sont dans dataframe'produit' mais pas dans dataframe 'transaction'
right_only = product_trans.loc[product_trans['_merge']=='right_only']
display(right_only)
display(right_only.shape)

,id_prod,date,session_id,client_id,price,categ,_merge
679332,0_1016,NaT,NaN,NaN,35.06,0,right_only
679333,0_1780,NaT,NaN,NaN,1.67,0,right_only
679334,0_1062,NaT,NaN,NaN,20.08,0,right_only
679335,0_1119,NaT,NaN,NaN,2.99,0,right_only
679336,0_1014,NaT,NaN,NaN,1.15,0,right_only
679337,1_0,NaT,NaN,NaN,31.82,1,right_only
679338,0_1318,NaT,NaN,NaN,20.92,0,right_only
679339,0_1800,NaT,NaN,NaN,22.05,0,right_only
679340,0_1645,NaT,NaN,NaN,2.99,0,right_only
679341,0_322,NaT,NaN,NaN,2.99,0,right_only


(21, 7)

**Observervation**: 
Nous avons 21 produits qui n'ont pas été vendus, avec des prix très variables. Ce serait judicieux de faire une promotion sur ces produits.

### Jointure interne des trois dataframes

In [27]:
# Inner merge des dataframes 'customer', 'transaction' et product
df = pd.merge( (pd.merge(customers, transaction, on='client_id')), product, on='id_prod' )
df.head()

,client_id,sex,birth,id_prod,date,session_id,price,categ
0,c_4410,f,1967,0_1316,2021-12-29 09:11:18.860592,s_141762,7.2,0
1,c_4410,f,1967,0_1316,2022-03-29 09:11:18.860592,s_186233,7.2,0
2,c_4410,f,1967,0_1316,2022-05-29 09:11:18.860592,s_215758,7.2,0
3,c_3654,f,1978,0_1316,2021-11-26 03:51:34.068872,s_125143,7.2,0
4,c_3654,f,1978,0_1316,2022-08-26 03:51:34.068872,s_258546,7.2,0


In [28]:
# Création de la variable 'age'

df['age']= (pd.DatetimeIndex(df['date']).year)-df['birth']

In [31]:
df.to_csv('df.csv')
display(df.nunique())
display(df.dtypes)
display(df)

client_id       8600
sex                2
birth             76
id_prod         3266
date          679332
session_id    342315
price           1443
categ              3
age               78
dtype: int64

client_id             object
sex                   object
birth                  int64
id_prod               object
date          datetime64[ns]
session_id            object
price                float64
categ                 object
age                    int64
dtype: object

,client_id,sex,birth,id_prod,date,session_id,price,categ,age
0,c_4410,f,1967,0_1316,2021-12-29 09:11:18.860592,s_141762,7.20,0,54
1,c_4410,f,1967,0_1316,2022-03-29 09:11:18.860592,s_186233,7.20,0,55
2,c_4410,f,1967,0_1316,2022-05-29 09:11:18.860592,s_215758,7.20,0,55
3,c_3654,f,1978,0_1316,2021-11-26 03:51:34.068872,s_125143,7.20,0,43
4,c_3654,f,1978,0_1316,2022-08-26 03:51:34.068872,s_258546,7.20,0,44
...,...,...,...,...,...,...,...,...,...
679327,c_8138,f,1984,0_394,2021-11-09 09:02:38.299240,s_116986,2.14,0,37
679328,c_8138,f,1984,0_394,2022-08-09 09:02:38.299240,s_250653,2.14,0,38
679329,c_8138,f,1984,0_394,2022-03-09 09:02:38.299240,s_176587,2.14,0,38
679330,c_8327,m,1972,0_394,2022-06-28 22:44:11.200205,s_230707,2.14,0,50
